In [1]:
import pandas as pandas
import nltk
from nltk.corpus import stopwords
import time

filename = 'train_data.csv'
t1 = time.time()
data = pandas.read_csv('train_data.csv')

In [7]:
data.head()

,stars,name,text,date,city,longitude,latitude,categories
0,1,McDonald's,Seriously cannot stand this McDonald's. They N...,2014-12-29,Glendale,-112.205020,33.509597,"['Burgers', 'Fast Food', 'Restaurants']"
1,5,Tom Colicchio's Craftsteak,"Amazing food, truly excellent best lobster bis...",2013-03-07,Las Vegas,-115.169751,36.102918,"['Steakhouses', 'Restaurants', 'Cheesesteaks',..."
2,5,Fishman Lobster Clubhouse Restaurant,"This was my second time here, and the seafood ...",2015-11-24,Toronto,-79.300795,43.824234,"['Seafood', 'Restaurants', 'Chinese', 'Live/Ra..."
3,1,Bonjour Brioche,"Long story short.\n\nBunch of rude, heartless,...",2016-12-20,Toronto,-79.346287,43.659795,"['Breakfast & Brunch', 'French', 'Restaurants']"
4,4,Dilly's Deli,We grabbed some dinner here last night before ...,2010-09-28,Tempe,-111.945365,33.422175,"['Caterers', 'Sandwiches', 'Event Planning & S..."


In [10]:
data_5=data[data["stars"]==5]
data_txt5=data_5["text"]
data_txt5=data_txt5[0:50]
data_txt5.to_csv("5star.csv",sep=",")

data_1=data[data["stars"]==1]
data_txt1=data_1["text"]
data_txt1=data_txt1[3:53]
data_txt1.to_csv("1star.csv",sep=",")